<a href="https://colab.research.google.com/github/sahil261005/API-Development-with-DRF/blob/main/Hackaton_Sahil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import json
import re

#  STEP 1: LOAD ORDERS
df_orders = pd.read_csv('orders.csv')
print(" Orders loaded.")

#  STEP 2: LOAD USERS
with open('users.json', 'r') as f:
    df_users = pd.DataFrame(json.load(f))
print(" Users loaded.")

# STEP 3: LOAD RESTAURANTS (Corrected Parser)
try:
    with open('restaurants.sql', 'r') as f:
        sql_content = f.read()

    # This regex now correctly looks for patterns like (1, 'Pizza Hut', 'Italian', 4.5) (without city)
    pattern = r"\(\s*(\d+)\s*,\s*'([^']*)'\s*,\s*'([^']*)'\s*,\s*([\d\.]+)\s*\)"
    data = re.findall(pattern, sql_content)

    # Convert to DataFrame with correct column names (without city)
    df_restaurants = pd.DataFrame(data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])

    # Fix data types (convert ID and Rating to numbers)
    df_restaurants['restaurant_id'] = pd.to_numeric(df_restaurants['restaurant_id'])
    df_restaurants['rating'] = pd.to_numeric(df_restaurants['rating'])

    print(" Restaurants loaded successfully!")
except Exception as e:
    print(f" Still having trouble: {e}")

#  STEP 4: MERGE THE DATA
if 'df_restaurants' in locals():
    merged_df = pd.merge(df_orders, df_users, on='user_id', how='left')
    final_df = pd.merge(merged_df, df_restaurants, on='restaurant_id', how='left')

    # --- STEP 5: SAVE FINAL FILE ---
    final_df.to_csv('final_food_delivery_dataset.csv', index=False)
    print("\n Final dataset created successfully!")
    print(final_df.head())
else:
    print(" Cannot merge because df_restaurants is missing.")

 Orders loaded.
 Users loaded.
 Restaurants loaded successfully!

 Final dataset created successfully!
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Re

# Analyze Gold Members Revnue

In [24]:
import pandas as pd
gold_members_df = final_df[final_df['membership'] == 'Gold']

# 3. Calculate the sum of the 'total_amount' for each city group.
city_revenue = gold_members_df.groupby('city')['total_amount'].sum().reset_index()


highest_revenue_city = city_revenue.loc[city_revenue['total_amount'].idxmax()]


print("\n Gold Member Revenue Analysis:")
print(f"The city with the highest 'Gold' member revenue is {highest_revenue_city['city']} with a total revenue of {highest_revenue_city['total_amount']:.2f}.")
print("\nTotal 'Gold' member revenue by city:")
print(city_revenue.sort_values(by='total_amount', ascending=False).to_string(index=False))


 Gold Member Revenue Analysis:
The city with the highest 'Gold' member revenue is Chennai with a total revenue of 1080909.79.

Total 'Gold' member revenue by city:
     city  total_amount
  Chennai    1080909.79
     Pune    1003012.32
Bangalore     994702.59
Hyderabad     896740.19


## Analyze Cuisine Average Order Value



In [31]:
import pandas as pd


if final_df.empty:
    print(" Your final_df is empty! Check your merge step.")
else:
    # 2. Look for the right column name automatically
    possible_cols = ['total_amount', 'order_amount', 'amount']
    actual_col = next((c for c in possible_cols if c in final_df.columns), None)

    if not actual_col:
        print(f" Could not find a revenue column. Columns available are: {final_df.columns.tolist()}")
    else:
        # Filter out rows where 'cuisine' is NaN before grouping
        df_filtered_cuisine = final_df.dropna(subset=['cuisine'])

        if df_filtered_cuisine.empty:
            print(" No valid cuisine data found after filtering. Check your merge and data quality!")
        else:
            cuisine_avg_order = df_filtered_cuisine.groupby('cuisine')[actual_col].mean().reset_index()

            if cuisine_avg_order.empty:
                print(" No cuisine data found after grouping. This should not happen if previous checks passed and data exists.")
            else:
                # 4. Find the highest
                idx = cuisine_avg_order[actual_col].idxmax()
                highest_avg_cuisine = cuisine_avg_order.loc[idx]

                print(f"--- Analysis using column: {actual_col} ---")
                print(f"The cuisine with the highest average is '{highest_avg_cuisine['cuisine']}' with {highest_avg_cuisine[actual_col]:.2f}")

                print("\nFull Rankings:")
                print(cuisine_avg_order.sort_values(by=actual_col, ascending=False).round(2))

--- Analysis using column: total_amount ---
The cuisine with the highest average is 'Mexican' with 808.02

Full Rankings:
   cuisine  total_amount
3  Mexican        808.02
2  Italian        799.45
1   Indian        798.47
0  Chinese        798.39


## Analyze High-Value Users

.


In [32]:
import pandas as pd


user_total_orders = final_df.groupby('user_id')['total_amount'].sum().reset_index()


high_value_users = user_total_orders[user_total_orders['total_amount'] > 1000]


num_high_value_users = high_value_users['user_id'].nunique()


print(f"\n Number of users with cumulative order value exceeding ₹1000: {num_high_value_users}")
print("\nHigh-value users by total amount:")
print(high_value_users.sort_values(by='total_amount', ascending=False).head().to_string(index=False))



 Number of users with cumulative order value exceeding ₹1000: 2544

High-value users by total amount:
 user_id  total_amount
    1515      11556.49
     650      10747.44
     496       9634.30
    2586       9486.61
    2615       9237.32


## Analyze Revenue by Restaurant Rating Range



In [33]:
import pandas as pd

# 1. Define rating ranges  and labels
bins = [0, 3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['<3.0', '3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0']


final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels, right=True, include_lowest=True)


revenue_by_rating_range = final_df.groupby('rating_range')['total_amount'].sum().reset_index()


highest_revenue_range = revenue_by_rating_range.loc[revenue_by_rating_range['total_amount'].idxmax()]


print("\n Revenue Analysis by Restaurant Rating Range:")
print(f"The rating range with the highest total revenue is '{highest_revenue_range['rating_range']}' with a total revenue of {highest_revenue_range['total_amount']:.2f}.")

print("\nTotal revenue by rating range (sorted descending):")
print(revenue_by_rating_range.sort_values(by='total_amount', ascending=False).round(2).to_string(index=False))


 Revenue Analysis by Restaurant Rating Range:
The rating range with the highest total revenue is '4.6-5.0' with a total revenue of 2197030.75.

Total revenue by rating range (sorted descending):
rating_range  total_amount
     4.6-5.0    2197030.75
     4.1-4.5    1960326.26
     3.0-3.5    1881754.57
     3.6-4.0    1717494.41
        <3.0     255018.13


/tmp/ipython-input-1132257747.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  revenue_by_rating_range = final_df.groupby('rating_range')['total_amount'].sum().reset_index()


## Analyze Gold Member Average Order Value by City



In [34]:
import pandas as pd

gold_members_df = final_df[final_df['membership'] == 'Gold']

# 2. Group the gold_members_df by the user's city ('city') and calculate the mean of the 'total_amount' for each city.
gold_city_avg_order = gold_members_df.groupby('city')['total_amount'].mean().reset_index()

# 3. Identify the city with the highest average order value from the gold_city_avg_order DataFrame.
highest_avg_order_city = gold_city_avg_order.loc[gold_city_avg_order['total_amount'].idxmax()]


print("\n Gold Member Average Order Value Analysis:")
print(f"The city with the highest average order value for 'Gold' members is '{highest_avg_order_city['city']}' with an average order value of {highest_avg_order_city['total_amount']:.2f}.")


print("\nAverage order value for 'Gold' members by city (sorted descending):")
print(gold_city_avg_order.sort_values(by='total_amount', ascending=False).round(2).to_string(index=False))


 Gold Member Average Order Value Analysis:
The city with the highest average order value for 'Gold' members is 'Chennai' with an average order value of 808.46.

Average order value for 'Gold' members by city (sorted descending):
     city  total_amount
  Chennai        808.46
Hyderabad        806.42
Bangalore        793.22
     Pune        781.16


## Analyze Cuisine Restaurant Count and Revenue




In [35]:
import pandas as pd


cuisine_restaurant_count = final_df.groupby('cuisine')['restaurant_id'].nunique().reset_index()
cuisine_restaurant_count.rename(columns={'restaurant_id': 'restaurant_count'}, inplace=True)


cuisine_total_revenue = final_df.groupby('cuisine')['total_amount'].sum().reset_index()
cuisine_total_revenue.rename(columns={'total_amount': 'total_revenue'}, inplace=True)


cuisine_analysis = pd.merge(cuisine_restaurant_count, cuisine_total_revenue, on='cuisine', how='left')

# 4. Print the resulting DataFrame, sorted as required
print("\n Cuisine Restaurant Count and Revenue Analysis:")
print("Cuisine with lowest restaurant count and their revenue contribution (sorted by restaurant count ascending, then total revenue descending):")
print(cuisine_analysis.sort_values(by=['restaurant_count', 'total_revenue'], ascending=[True, False]).round(2).to_string(index=False))


 Cuisine Restaurant Count and Revenue Analysis:
Cuisine with lowest restaurant count and their revenue contribution (sorted by restaurant count ascending, then total revenue descending):
cuisine  restaurant_count  total_revenue
Chinese               120     1930504.65
Italian               126     2024203.80
 Indian               126     1971412.58
Mexican               128     2085503.09


## Analyze Gold Member Order Percentage




In [36]:
import pandas as pd


total_orders = final_df.shape[0]

gold_member_orders_df = final_df[final_df['membership'] == 'Gold']

gold_member_order_count = gold_member_orders_df.shape[0]

# 4. Calculate the percentage of orders contributed by 'Gold' members.
percentage_gold_orders = (gold_member_order_count / total_orders) * 100

# 5. Round the calculated percentage to the nearest integer.
rounded_percentage = round(percentage_gold_orders)

print(f"\n Total number of orders: {total_orders}")
print(f" Number of orders by 'Gold' members: {gold_member_order_count}")
print(f" Percentage of total orders by 'Gold' members: {rounded_percentage:.0f}%")


 Total number of orders: 10000
 Number of orders by 'Gold' members: 4987
 Percentage of total orders by 'Gold' members: 50%


## Analyze Restaurant Average Order Value (Low Order Count)




In [37]:
import pandas as pd


restaurant_order_counts = final_df.groupby('restaurant_id')['order_id'].count().reset_index()
restaurant_order_counts.rename(columns={'order_id': 'order_count'}, inplace=True)

low_order_restaurants_ids = restaurant_order_counts[restaurant_order_counts['order_count'] < 20]


low_order_restaurants_df = pd.merge(final_df, low_order_restaurants_ids[['restaurant_id']], on='restaurant_id', how='inner')

low_order_avg_value = low_order_restaurants_df.groupby('restaurant_id')['total_amount'].mean().reset_index()

highest_avg_restaurant = low_order_avg_value.loc[low_order_avg_value['total_amount'].idxmax()]

print("\n Analysis of Restaurants with Low Order Count:")
print(f"The restaurant with fewer than 20 orders and the highest average order value is Restaurant ID {highest_avg_restaurant['restaurant_id']} with an average of {highest_avg_restaurant['total_amount']:.2f}.")

print("\nAverage order value for restaurants with fewer than 20 orders (sorted descending):")
print(low_order_avg_value.sort_values(by='total_amount', ascending=False).round(2).to_string(index=False))


 Analysis of Restaurants with Low Order Count:
The restaurant with fewer than 20 orders and the highest average order value is Restaurant ID 294.0 with an average of 1040.22.

Average order value for restaurants with fewer than 20 orders (sorted descending):
 restaurant_id  total_amount
           294       1040.22
           262       1029.47
            77       1029.18
           193       1026.31
             7       1002.14
           298        989.82
            56        989.47
           135        988.70
           343        986.03
           312        982.73
           160        977.84
           380        973.17
           456        973.13
           118        965.30
           158        964.58
           374        954.51
           189        954.05
           360        948.22
           323        942.27
           179        941.89
           248        941.33
            98        940.72
           385        938.20
           325        934.93
           365 

In [38]:
restaurant_294_details = df_restaurants[df_restaurants['restaurant_id'] == 294]
display(restaurant_294_details)

,restaurant_id,restaurant_name,cuisine,rating
293,294,Restaurant_294,Italian,3.1


## Analyze Highest Revenue Combination



**Reasoning**:
I will now implement the steps outlined in the reasoning and instructions to calculate and compare the total revenue for the specified membership and cuisine combinations, storing the results, and identifying the highest revenue combination.



In [39]:
import pandas as pd


revenue_combinations = {}

revenue_combinations['Gold_Indian'] = final_df[
    (final_df['membership'] == 'Gold') & (final_df['cuisine'] == 'Indian')
]['total_amount'].sum()


revenue_combinations['Gold_Italian'] = final_df[
    (final_df['membership'] == 'Gold') & (final_df['cuisine'] == 'Italian')
]['total_amount'].sum()

revenue_combinations['Regular_Indian'] = final_df[
    (final_df['membership'] == 'Regular') & (final_df['cuisine'] == 'Indian')
]['total_amount'].sum()

revenue_combinations['Regular_Chinese'] = final_df[
    (final_df['membership'] == 'Regular') & (final_df['cuisine'] == 'Chinese')
]['total_amount'].sum()


revenue_series = pd.Series(revenue_combinations)

highest_revenue_combination = revenue_series.idxmax()
highest_revenue_amount = revenue_series.max()

print("\n Highest Revenue Combination Analysis:")
print(f"The combination with the highest revenue is '{highest_revenue_combination}' with a total revenue of {highest_revenue_amount:.2f}.")


print("\nTotal revenue for specified combinations (sorted descending):")
print(revenue_series.sort_values(ascending=False).round(2).to_string())


 Highest Revenue Combination Analysis:
The combination with the highest revenue is 'Gold_Italian' with a total revenue of 1005779.05.

Total revenue for specified combinations (sorted descending):
Gold_Italian       1005779.05
Regular_Indian      992100.27
Gold_Indian         979312.31
Regular_Chinese     952790.91


## Analyze Quarterly Revenue


In [40]:
import pandas as pd

# 1. Convert 'order_date' to datetime objects
final_df['order_date'] = pd.to_datetime(final_df['order_date'], format='%d-%m-%Y', errors='coerce')


final_df['quarter'] = final_df['order_date'].dt.quarter


quarterly_revenue = final_df.groupby('quarter')['total_amount'].sum().reset_index()


highest_revenue_quarter = quarterly_revenue.loc[quarterly_revenue['total_amount'].idxmax()]

# 5. Print the quarter with the highest total revenue and its corresponding amount
print("\n Quarterly Revenue Analysis:")
print(f"The quarter with the highest total revenue is Quarter {highest_revenue_quarter['quarter']} with a total revenue of {highest_revenue_quarter['total_amount']:.2f}.")


print("\nTotal revenue by quarter (sorted descending):")
print(quarterly_revenue.sort_values(by='total_amount', ascending=False).round(2).to_string(index=False))


 Quarterly Revenue Analysis:
The quarter with the highest total revenue is Quarter 3.0 with a total revenue of 2037385.10.

Total revenue by quarter (sorted descending):
 quarter  total_amount
       3    2037385.10
       4    2018263.66
       1    2010626.64
       2    1945348.72




**1. Gold Member Revenue by City:**

```
     city  total_amount
  Chennai    1080909.79
     Pune    1003012.32
Bangalore     994702.59
Hydrabad     896740.19
```

**2. Cuisine Average Order Value:**

```
cuisine  total_amount
Mexican        808.02
Italian        799.45
 Indian        798.47
Chinese        798.39
```



**4. Revenue by Restaurant Rating Range:**
*
```
rating_range  total_amount
     4.6-5.0    2197030.75
     4.1-4.5    1960326.26
     3.0-3.5    1881754.57
     3.6-4.0    1717494.41
        <3.0     255018.13
```

**5. Gold Member Average Order Value by City:**
*
     city  total_amount
  Chennai        808.46
Hydrabad        806.42
Bangalore        793.22
     Pune        781.16
```

**6. Cuisine Restaurant Count and Revenue:**

cuisine  restaurant_count  total_revenue
Chinese               120     1930504.65
Italian               126     2024203.80
 Indian               126     1971412.58
Mexican               128     2085503.09
```

**7. Gold Member Order Percentage:**

*    'Gold' members contribute **50%** of the total orders.
*    Total orders: 10000, Gold member orders: 4987.

**8. Restaurant Average Order Value (Low Order Count):**
 Restaurant ID **294** has fewer than 20 orders and the highest average order value of **₹1040.22**.

**9. Highest Revenue Combination:**

```
Gold_Italian       1005779.05
Regular_Indian      992100.27
Gold_Indian         979312.31
Regular_Chinese     952790.91
```

**10. Quarterly Revenue:**
```
 quarter  total_amount
       3    2037385.10
       4    2018263.66
       1    2010626.64
       2    1945348.72
```

## Calculate Total Revenue from Hyderabad




In [41]:
import pandas as pd


hyderabad_orders_df = final_df[final_df['city'] == 'Hyderabad']


total_revenue_hyderabad = hyderabad_orders_df['total_amount'].sum()


print(f" Total revenue from orders placed by users in Hyderabad: ₹{total_revenue_hyderabad:.2f}")

 Total revenue from orders placed by users in Hyderabad: ₹1889366.58


## Count Orders for High-Rated Restaurants



In [42]:
import pandas as pd


high_rated_restaurants_orders_df = final_df[final_df['rating'] >= 4.5]


# This count represents the total number of orders from restaurants with a rating of 4.5 or higher.
orders_from_high_rated_restaurants = high_rated_restaurants_orders_df.shape[0]

# 3. Print the resulting count.
print(f" Number of orders from restaurants with a rating of 4.5 or higher: {orders_from_high_rated_restaurants}")

 Number of orders from restaurants with a rating of 4.5 or higher: 3374


## Identify Top Gold Member Revenue City



In [22]:
print(f"The city with the highest 'Gold' member revenue is: {highest_revenue_city['city']}")

The city with the highest 'Gold' member revenue is: Chennai


In [43]:
import pandas as pd

# 1. Get the city name from the highest_revenue_city Series
top_gold_revenue_city = highest_revenue_city['city']


gold_members_top_city_orders = final_df[
    (final_df['membership'] == 'Gold') &
    (final_df['city'] == top_gold_revenue_city)
]

num_orders_gold_top_city = gold_members_top_city_orders.shape[0]

# 4. Print the result
print(f" Number of orders placed by 'Gold' members in {top_gold_revenue_city}: {num_orders_gold_top_city}")

 Number of orders placed by 'Gold' members in Chennai: 1337
